# F1 Fantasy Program

## ReadMe

The goal of this program is to webscrape, and gather, F1 data. The data would then be saved, transformed, and analysed.

# Imports

#### FastF1

In [54]:
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests
import glob

import fastf1

In [55]:
from datetime import datetime

#### MySQL

In [56]:
import mysql.connector
import json

In [57]:
import glob
import csv

### Ignore Warnings

In [58]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# FastF1 Data Aquisition

### Enable Caching

In [59]:
fastf1.Cache.enable_cache('cache')

### Get the completed events (race weekends) of the season (year)

In [60]:
def get_completed_events(season, b_print=False):
    # Retrieve the event schedule for the given season using the fastf1 library
    schedule = fastf1.get_event_schedule(season)
    
    # Initialize an empty list to store completed events
    completed_events = []

    # Loop through the round numbers in the event schedule, skipping the first one (index 0)
    for event in schedule['RoundNumber'][1:]:
        # Check if the event date is in the past (i.e., the event is completed)
        if schedule['EventDate'][event] < datetime.now():
            # Append the completed event to the completed_events list
            completed_events.append(schedule.iloc[[event]])

    # Return the list of completed events
    return completed_events
	
# Commented by GPT4

### Get Amount of completed events of the season (year)

In [65]:
def get_completed_events_count(season):
	count = len(get_completed_events(season))
	return count

### Load the data of a completed session

In [61]:
def get_session_data(season, event_num, session_type):
    df_session = fastf1.get_session(season, event_num, session_type)
    df_session.load()
    return df_session

### Load the data of all the completed sessions

In [53]:
def get_completed_sessions(season, completed_events, session_type):
    # Initialize an empty list to store completed sessions
    completed_sessions = []

    # Loop through the completed_events list, using the enumerate function to keep track of the index
    for i, event in enumerate(completed_events):
        # Get the session data for the current event and append it to the list of completed sessions
        session = get_session_data(season, event['RoundNumber'], session_type)
        completed_sessions.append(session)

    # Return the list of completed sessions
    return completed_sessions

# commented by GPT3.5

### Get All Completed Sessions' data

In [36]:
def get_completed_sessions(season, completed_events, session_type):
    # Returns a list of dataframes
    completed_sessions = []

    for event_df in completed_events:
        for _, event in event_df.iterrows():
            session = get_session_data(season, event['RoundNumber'], session_type)
            completed_sessions.append(session)

    return completed_sessions


### Get All Outstanding Sessions' data

###### Get the data of the sessions that have not yet been collected.

### Save sessions to csv

In [38]:
def save_df_list_to_csv(dataframe_list, name="dataframe"):
    for i, dataframe in enumerate(dataframe_list):
        file_name = name + '_' + str(i) + '.csv'
        dataframe.to_csv(file_name)

### Log Loaded Race

In [43]:
loaded_races_logfile = 'loaded_races_logfile.txt'

In [44]:
def log_loaded_race(race_number):
	timestamp_format = '%Y-%h-%d-%H:%M:%S' #Year-Monthname-Day-Hour-Minute-Second
	now = datetime.now()
	timestamp = now.strftime(timestamp_format)
	with open(loaded_races_logfile, 'a') as log:
		log.write(timestamp + ','+race_number+'\n')

### Read logfile

In [45]:
def read_loaded_races_logfile():
	loaded_races = []
	with open(loaded_races_logfile, 'r') as log:
		for line in log.readlines():
			fields = line.strip().split(',')
			loaded_races.append(fields[1])
	
	return loaded_races

### Find missing races

In [46]:
def find_missing_races(races_list):
	# given a list of races, it finds the numbers that are missing
	missing = set(range(min(races_list), max(races_list)+1)) - set(races_list)
	return missing

# Webscraping

In [39]:
url = 'https://www.f1fantasytracker.com/prices.html'  # replace with the URL of the website you want to scrape
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

current_price = soup.find('span', {'id': 'CurrentPrice1'}).text
season_price = soup.find('span', {'id': 'SeasonPrice1'}).text

print(current_price)
print(season_price)

# Save Data into CSV files

In [ ]:

# const parameters for api requests
season = 2023
session_type = 'R'

event_number = 0

with open('loaded_races_log.txt', 'a+') as log_file:
	for lines in log_file.readlines():
		

In [ ]:
results_list = [session.results for session in ]
save_df_list_to_csv(results_list)
print(results_list)

# MySQL f1stats

### Connect to Database

###### Create a 'private_config.json' file, and add the details of your database there.

In [42]:
with open('private_config.json', 'r') as file:
	config_data = json.load(file)


mydb = mysql.connector.connect(
  host=config_data['db_host'],
  user=config_data['db_user'],
  password=config_data['db_password'],
  database=config_data['db_database']
)

mycursor = mydb.cursor()

### Load data into Database